In [68]:
from wiley_tdm import TDMClient
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re
import pandas as pd
import unicodedata
from IPython.display import clear_output

In [10]:
# this part is irrelevant
os.environ["TDM_API_TOKEN"] = input("TDM API TOKEN:\n")
tdm = TDMClient()
tdm.download_pdf("10.1002/14651858.CD010346.pub2")

DownloadResult(doi='10.1002/14651858.CD010346.pub2', status=<DownloadStatus.SUCCESS: 1>, comment='', path=WindowsPath('downloads/10.1002-14651858.CD010346.pub2.pdf'), size=1147, duration=1.824482199997874, api_status=<HTTPStatus.OK: 200>)

In [88]:
HTML = '<select class="search-select-topic-list hw-40"> <option value="Allergy &amp; intolerance">Allergy &amp; intolerance</option><option value="Blood disorders">Blood disorders</option><option value="Cancer">Cancer</option><option value="Child health">Child health</option><option value="Complementary &amp; alternative medicine">Complementary &amp; alternative medicine</option><option value="Consumer &amp; communication strategies">Consumer &amp; communication strategies</option><option value="Dentistry &amp; oral health">Dentistry &amp; oral health</option><option value="Developmental, psychosocial &amp; learning problems">Developmental, psychosocial &amp; learning problems</option><option value="Diagnosis">Diagnosis</option><option value="Ear, nose &amp; throat">Ear, nose &amp; throat</option><option value="Effective practice &amp; health systems">Effective practice &amp; health systems</option><option value="Endocrine &amp; metabolic">Endocrine &amp; metabolic</option><option value="Eyes &amp; vision">Eyes &amp; vision</option><option value="Gastroenterology &amp; hepatology">Gastroenterology &amp; hepatology</option><option value="Genetic disorders">Genetic disorders</option><option value="Gynaecology">Gynaecology</option><option value="Health &amp; safety at work">Health &amp; safety at work</option><option value="Health professional education">Health professional education</option><option value="Heart &amp; circulation">Heart &amp; circulation</option><option value="Infectious disease">Infectious disease</option><option value="Insurance medicine">Insurance medicine</option><option value="Kidney disease">Kidney disease</option><option value="Lungs &amp; airways">Lungs &amp; airways</option><option value="Mental health">Mental health</option><option value="Methodology">Methodology</option><option value="Neonatal care">Neonatal care</option><option value="Neurology">Neurology</option><option value="Orthopaedics &amp; trauma">Orthopaedics &amp; trauma</option><option value="Pain &amp; anaesthesia">Pain &amp; anaesthesia</option><option value="Pregnancy &amp; childbirth">Pregnancy &amp; childbirth</option><option value="Public health">Public health</option><option value="Reproductive &amp; sexual health">Reproductive &amp; sexual health</option><option value="Rheumatology">Rheumatology</option><option value="Skin disorders">Skin disorders</option><option value="Tobacco, drugs &amp; alcohol">Tobacco, drugs &amp; alcohol</option><option value="Urology">Urology</option><option value="Wounds">Wounds</option></select>'
terms = re.findall(r"value=\".*?\"", HTML)
cochrane_topics = [tt[7:-1].replace("&amp;", "&") for tt in terms]

def search_cochrane_js(term="", by_topic=None):
    # by_topic: should be one of the cochrane_topics
    options = Options()
    options.add_argument('--headless')  # Run without GUI
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)

    driver.get("https://www.cochranelibrary.com/advanced-search")

    # Find the main search input and enter your term
    search_box = driver.find_element(By.NAME, "searchText")
    search_box.send_keys(term)

    if by_topic!=None:
        if by_topic not in cochrane_topics:
            print("Invalid topic! Available topics are: "+str(cochrane_topics))
        else:
            search_type_dropdown = Select(driver.find_element(By.ID, "searchOptions0"))
            search_type_dropdown.select_by_value("13") # value 13 means "Cochrane topic"
            topic_dropdown = Select(driver.find_element(By.CSS_SELECTOR, "select.search-select-topic-list"))
            topic_dropdown.select_by_visible_text(by_topic)
            # driver.save_screenshot("after_dropdown_selection.png")    

    # Submit the form
    search_box.submit()
    time.sleep(5)  # Wait for JS to load results
    
    # Extract search results
    results = []
    next_link_el = "start"
    while next_link_el:
        items = driver.find_elements(By.CSS_SELECTOR, ".result-title a")
        for item in items:
            title = item.text
            doi = item.get_attribute("href")
            print(title)
            results += [{"title":title, "URL":doi}]

        try:
            next_link_el = driver.find_element(By.CSS_SELECTOR, ".pagination-next-link a")
            next_url = next_link_el.get_attribute("href")
            driver.get(next_url)
            time.sleep(3)
        except:
            print("Reached the last page of the search results.")
            break

    driver.quit()
    return results

In [83]:
results = search_cochrane_js("", by_topic="Mental health")
for r in results:
    print(r)

Day hospital versus admission for acute psychiatric disorders
Psychological therapies for the treatment of depression in chronic obstructive pulmonary disease
Folic acid with or without vitamin B12 for the prevention and treatment of healthy elderly and demented people
Perphenazine for schizophrenia
Enhanced rehabilitation and care models for adults with dementia following hip fracture surgery
Compliance therapy for schizophrenia
Monetary incentives for schizophrenia
'As required' medication regimens for seriously mentally ill people in hospital
Specialised early intervention teams (extended time) for recent‐onset psychosis
Specialised early intervention teams for recent‐onset psychosis
Interventions for treating anxiety after stroke
'Third wave' cognitive and behavioural therapies versus other psychological therapies for depression
Risperidone for psychosis‐induced aggression or agitation (rapid tranquillisation)
Calcium channel blockers for antipsychotic‐induced tardive dyskinesia
No

## Work with a list of review titles and URLs derived from the auto-search

In [87]:
df = pd.DataFrame(columns=["title","URL"])
for rr in results:
    df = pd.concat([df, pd.DataFrame({"title":rr["title"], "URL":rr["DOI"]}, index=[0])], ignore_index=True)
df.to_excel("SearchResults_topic_mental-health.xlsx", index=False)

In [ ]:
def clean_text(s):
    return re.sub(r'\W+', '', s)

def get_guideline_text_safe(driver, timeout=5):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: (
                (el := d.find_element(By.CSS_SELECTOR, 'a[title="Guidelines"]')) 
                and el.text.strip() != ''
            )
        )
        return driver.find_element(By.CSS_SELECTOR, 'a[title="Guidelines"]').text
    except (TimeoutException, NoSuchElementException):
        return ""

# an example of guideline cited review: https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD012347.pub2/full
# an example of not-cited review: https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD003153/full

df = pd.read_excel("SearchResults_topic_mental-health.xlsx")
# df = pd.DataFrame({"title":["Psychological therapies for the treatment of depression in chronic obstructive pulmonary disease",
#                             "D‐cycloserine for Alzheimer's disease"],
#                    "URL":["https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD012347.pub2/full",
#                           "https://www.cochranelibrary.com/cdsr/doi/10.1002/14651858.CD003153/full"]})
retry_limit = 50
df["guideline"] = 0
df["retry"] = 0
df["done"] = 0
options = Options()
options.add_argument('--headless')  # Run without GUI
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1920,1080")

for ii,row in df.iterrows():
    clear_output(wait=True)
    print(f"[{ii:03d}] {row["title"]}")
    page_retrieved = False
    retry = 0
    # driver.get(row["URL"])
    while not page_retrieved:
        driver = webdriver.Chrome(options=options)
        driver.get(row["URL"])
        title_element = re.search(r"{database.*?}}",driver.page_source)
        page_retrieved = title_element!=None
        if title_element == None:
            print(f"... Title element not found, retry")
            retry += 1
            time.sleep(3)
            driver.quit()
            continue
            
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[title="Guidelines"]')))
     
    gline_ct = driver.find_element(By.CSS_SELECTOR, 'a[title="Guidelines"]').text
    # gline_ct = get_guideline_text_safe(driver)
    driver.save_screenshot("debug_html.png")
    print(f"... {gline_ct}")
    if len(gline_ct)>0:
        df.loc[ii,"guideline"] = int(gline_ct.split(" ")[2])
    df.loc[ii,"retry"] = retry
    df.loc[ii,"done"] = int(page_retrieved)
    # print(gline_ct.text)
    # driver.delete_all_cookies()
    # driver.execute_script("window.localStorage.clear(); window.sessionStorage.clear();")
    # driver.get("about:blank")
    driver.quit()
    time.sleep(3) # to avoid being regarded as a DDoS attack
    if ii%5==4:
        df.to_excel("SearchResults_topic_mental-health_guideline.xlsx", index=False)
# print(df)

[005] Compliance therapy for schizophrenia
... Cited in 1 guideline


In [ ]:
a = re.search(r"hahaha", "adfasrejsdfl;ja;ehaHahaafjdsfasdf")
print(a)

In [ ]:
<a href="/cdsr/doi/10.1002/14651858.CD012347.pub2/related-content#guidelines_data" title="Guidelines" style="display: inline">Cited in 11 guidelines</a>